In [10]:
from splinter import browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# New York Regions

In [11]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

url = "https://commons.wikimedia.org/wiki/File:Map_of_New_York_Economic_Regions.svg"
browser.visit(url)

time.sleep(1)

html = browser.html
soup = bs(html, "html.parser")



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\hk_la\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [12]:
region_list = soup.find("div", class_="description")

In [13]:
region_list = region_list.findAll("li")

In [14]:
USRegions_df = []
counter = 0
for region in region_list:
    counter = 1 + counter
    region_name = region.find("a").text
#     print(region.findAll("a"))
    numRegion = len(region.findAll("a"))
    skip_first = 0
    for i in range(numRegion):
        if skip_first < numRegion-1:
            skip_first = i + 1
            counties = region.findAll("a")[skip_first].text
            data = {'Region': region_name, 
                    'Counties': counties}
            USRegions_df.append(data)


In [15]:
# USRegions_df = pd.DataFrame(USRegions_df)

In [16]:
browser.quit()

In [17]:
USRegions_df

[{'Region': 'Western New York', 'Counties': 'Orleans'},
 {'Region': 'Western New York', 'Counties': 'Genesee'},
 {'Region': 'Western New York', 'Counties': 'Wyoming'},
 {'Region': 'Western New York', 'Counties': 'Niagara'},
 {'Region': 'Western New York', 'Counties': 'Erie'},
 {'Region': 'Western New York', 'Counties': 'Chautauqua'},
 {'Region': 'Western New York', 'Counties': 'Cattaraugus'},
 {'Region': 'Western New York', 'Counties': 'Allegany'},
 {'Region': 'Finger Lakes', 'Counties': 'Monroe'},
 {'Region': 'Finger Lakes', 'Counties': 'Livingston'},
 {'Region': 'Finger Lakes', 'Counties': 'Wayne'},
 {'Region': 'Finger Lakes', 'Counties': 'Ontario'},
 {'Region': 'Finger Lakes', 'Counties': 'Yates'},
 {'Region': 'Finger Lakes', 'Counties': 'Seneca'},
 {'Region': 'Finger Lakes', 'Counties': 'Cayuga'},
 {'Region': 'Southern Tier', 'Counties': 'Steuben'},
 {'Region': 'Southern Tier', 'Counties': 'Schuyler'},
 {'Region': 'Southern Tier', 'Counties': 'Chemung'},
 {'Region': 'Southern Tier'

In [151]:
# Import Dependencies
import pandas as pd
import os, fnmatch
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager

def file_search():
    # Searching for csv filenames

    for root, dirs, files in os.walk("resources", topdown=False):
        resourcesVFiles = []
        resourcesRFiles = []
        resourcesTotFiles = []
        for name in files:
            if os.path.join(name).startswith("Volume"):
                resourcesVFiles.append(os.path.join(root, name))
            elif os.path.join(name).rfind("total"):
                resourcesTotFiles.append(os.path.join(root, name))
            else:
                resourcesRFiles.append(os.path.join(root, name))
    
    # Load all relevant filenames and path to a dictionary
    resourcesFile = {
        "volume_data": resourcesVFiles,
        "region_data": resourcesRFiles,
        "sales_tot_data": resourcesTotFiles
    }
    return  resourcesFile


def volume_file():

    # Extract filenames and Path or csv files relating to volume unit

    resourcesVFiles = file_search()['volume_data']

    numfile = len(resourcesVFiles)

    # Load Volume into dfs
    for i in range(len(resourcesVFiles)):
        file = resourcesVFiles[i]
        i = i + 1
        variable_name = f"vol_{i}"
        globals()[variable_name] =  pd.read_csv(file)

    # Union Join dfs
    if numfile == 3:
        volume_temp = pd.concat([vol_1, vol_2])
        volume = pd.concat([volume_temp, vol_3])
    elif numfile == 2: 
        volume = pd.concat([vol_1, vol_2])
    else:
        volume == vol_1

    # Replace N/A with zeros
    volume.fillna(0)

    for col in volume.columns:
        if not (col == 'Week' or col == 'Status' or col == 'Year'):
            try:
                volume[col] = volume[col].str.replace(",", "").astype(float)
            except AttributeError:
                print(f'skipped {col}')

    volume_transformed = volume.fillna(0)

    volume_transformed['WEDate'] = pd.to_datetime(volume_transformed.Week)

    actual_vol_temp = volume_transformed.loc[(volume_transformed["Status"] == "Actual"), :]
    actual_vol = actual_vol_temp.to_dict("records")

    return actual_vol

def scrape_region():
    # Identify Counties with their Regions
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)

    url = "https://www.britannica.com/topic/list-of-cities-and-towns-in-the-United-States-2023068"
    browser.visit(url)

    time.sleep(1)

    html = browser.html
    soup = bs(html, "html.parser")
    sections_id = 326620
    lst_section_id = 50
    USRegions_df = []
    # Loop starts
    for x in range(lst_section_id):
        section_id = sections_id + x
        section_name = f'ref{section_id}'
        section = soup.findAll("section", id=section_name)[0]
        states = section.find("h2", class_="h1").text
        for cities in range(len(section.findAll("li"))):
            row = section.findAll("li")[cities]
            try:
                row_text = row.find("a").text
            except AttributeError: 
                row_text = row.find("div").text
            data = {'state': states,
                    'city': row_text}
            USRegions_df.append(data)
    browser.quit()

    return USRegions_df



In [153]:
# def tot_file():

    # Extract filenames and Path or csv files relating to volume unit

resourcesTotFiles = file_search()['sales_tot_data']

numfile = len(resourcesTotFiles)

# Load Volume into dfs
for i in range(len(resourcesTotFiles)):
    file = resourcesTotFiles[i]
    i = i + 1
    variable_name = f"tot_{i}"
    globals()[variable_name] =  pd.read_csv(file)

if numfile == 3:
    sales_tot_temp = pd.concat([tot_1, tot_2])
    sales_tot = pd.concat([sales_tot_temp, tot_3])
elif numfile == 2: 
    sales_tot = pd.concat([tot_1, tot_2])
else:
    sales_tot == tot_1

sales_tot_transformed = sales_tot.fillna(0)





====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\hk_la\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


TypeError: Can only merge Series or DataFrame objects, a <class 'list'> was passed

In [ ]:
sales_tot_transformed[sales_tot_transformed['Region'].isna()]['Geography'].unique()

# https://www.britannica.com/topic/list-of-cities-and-towns-in-the-United-States-2023068

In [158]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

url = "https://www.britannica.com/topic/list-of-cities-and-towns-in-the-United-States-2023068"
browser.visit(url)

time.sleep(1)

html = browser.html
soup = bs(html, "html.parser")
sections_id = 326620
lst_section_id = 50
USRegions_df = []
# Loop starts
for x in range(lst_section_id):
    section_id = sections_id + x
    section_name = f'ref{section_id}'
    section = soup.findAll("section", id=section_name)[0]
    states = section.find("h2", class_="h1").text
    for cities in range(len(section.findAll("li"))):
        row = section.findAll("li")[cities]
        try:
            row_text = row.find("a").text
        except AttributeError: 
            row_text = row.find("div").text
        data = {'state': states,
                'city': row_text}
        USRegions_df.append(data)
USRegions_df = pd.DataFrame(USRegions_df)



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\hk_la\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [160]:
USRegions_df

,state,city
0,Alabama,Alexander City
1,Alabama,Andalusia
2,Alabama,Anniston
3,Alabama,Athens
4,Alabama,Atmore
...,...,...
1956,Wyoming,Sheridan
1957,Wyoming,Ten Sleep
1958,Wyoming,Thermopolis
1959,Wyoming,Torrington


In [162]:
sales_tot_transformed = sales_tot_transformed.merge(USRegions_df, left_on="Geography", right_on="city", how="left")

In [163]:
sales_tot_transformed

,Geography,Timeframe,Current Year Week Ending,Type,ASP Current Year,Total Bulk and Bags Units,4046 Units,4225 Units,4770 Units,TotalBagged Units,SmlBagged Units,LrgBagged Units,X-LrgBagged Units,Bulk GTIN,state,city
0,Albany,Weekly,2020-01-06 00:00:00,Conventional,1.155100,161726.12,3737.00,140642.19,184.16,10996.65,0.0,0.0,0.0,6166.13,Georgia,Albany
1,Albany,Weekly,2020-01-06 00:00:00,Conventional,1.155100,161726.12,3737.00,140642.19,184.16,10996.65,0.0,0.0,0.0,6166.13,New York,Albany
2,Albany,Weekly,2020-01-06 00:00:00,Conventional,1.155100,161726.12,3737.00,140642.19,184.16,10996.65,0.0,0.0,0.0,6166.13,Oregon,Albany
3,Albany,Weekly,2020-01-12 00:00:00,Conventional,1.227609,116738.30,3649.24,90717.11,203.78,16359.88,0.0,0.0,0.0,5808.29,Georgia,Albany
4,Albany,Weekly,2020-01-12 00:00:00,Conventional,1.227609,116738.30,3649.24,90717.11,203.78,16359.88,0.0,0.0,0.0,5808.29,New York,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16931,West,Weekly,2022-03-27 00:00:00,Organic,2.132444,279969.07,16833.47,19138.13,0.00,89663.57,0.0,0.0,0.0,154333.89,NaN,NaN
16932,Total U.S.,Weekly,2022-04-17 00:00:00,Organic,1.774314,2073388.73,98894.63,93306.07,412.89,1366905.16,0.0,0.0,0.0,513869.98,NaN,NaN
16933,Total U.S.,Weekly,2022-04-10 00:00:00,Organic,1.734692,2135495.11,84439.07,107442.51,315.94,1417980.89,0.0,0.0,0.0,525316.71,NaN,NaN
16934,Total U.S.,Weekly,2022-04-03 00:00:00,Organic,1.701004,2180929.28,73724.50,107346.93,306.77,1495281.89,0.0,0.0,0.0,504269.19,NaN,NaN
